[View in Colaboratory](https://colab.research.google.com/github/opooladz/ESPRIT-Autoencoder-Anomaly-Detection/blob/master/ESPRIT2.ipynb)

In [0]:
!apt-get install nvidia-cuda-toolkit
!pip3 install numba

import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"

from numba import cuda
import numpy as np
import time

@cuda.jit
def hello(data):
    data[cuda.blockIdx.x, cuda.threadIdx.x] = cuda.blockIdx.x

numBlocks = 5
threadsPerBlock = 10

data = np.ones((numBlocks, threadsPerBlock), dtype=np.uint8)

hello[numBlocks, threadsPerBlock](data)

print(data)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  accountsservice acpid activity-log-manager adwaita-icon-theme apg aspell
  aspell-en at-spi2-core avahi-daemon avahi-utils bbswitch-dkms bind9-host
  binfmt-support bluez bluez-obexd bsdmainutils ca-certificates-java
  cheese-common clang-3.8 cracklib-runtime crda cups-pk-helper dbus
  dbus-user-session dbus-x11 dconf-cli dconf-gsettings-backend dconf-service
  default-jre default-jre-headless desktop-file-utils dictionaries-common
  distro-info-data dkms dmsetup dns-root-data dnsmasq-base dosfstools eject
  emacsen-common enchant evolution-data-server evolution-data-server-common
  fakeroot file fontconfig fonts-dejavu-extra fuse gcr gdisk geoclue
  geoclue-ubuntu-geoip geoip-database gettext-base gir1.2-atk-1.0
  gir1.2-freedesktop gir1.2-gdkpixbuf-2.0 gir1.2-glib-2.0 gir1.2-gtk-3.0
  gir1.2-ibus-1.0 gir1.2-notify-0.7 gir1.2-packagekit

In [0]:
!pip3 install numba

    100% |████████████████████████████████| 1.9MB 9.4MB/s 
    100% |████████████████████████████████| 15.9MB 2.4MB/s 


In [0]:
import numpy as np
from scipy.linalg import toeplitz
from numpy import linalg as lg
from time import time
from typing import Tuple
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from numba import jit

flatten = lambda l: [item for sublist in l for item in sublist]

@jit
def compute_autocovariance(x: np.ndarray, M: int) -> np.ndarray:
    """
    This function compute the auto-covariance matrix of a numpy signal.
    The auto-covariance is computed as follows
    .. math:: \textbf{R}=\frac{1}{N}\sum_{M-1}^{N-1}\textbf{x}_{m}\textbf{x}_{m}^{H}
    where :math:`\textbf{x}_{m}^{T}=[x[m],x[m-1],x[m-M+1]]`.
    :param x: 1-D vector of size N
    :param M:  int, optional. Size of signal block.
    :returns: NxN ndarray
    """

    # Create covariance matrix for psd estimation
    # length of the vector x
    x = np.asarray(x).squeeze()
    assert x.ndim == 1, '1-D only'
    N = x.size

    # Create column vector (Nx1) from row array
    x_vect = x[None, :].T

    # init covariance matrix
    yn = x_vect[M-1::-1]  # reverse order from M-1 to 0

    R = yn @ yn.conj().T  # zeroth lag
    # about 5-8% of computation time
    for i in range(1, N-M):  # no zero because we just computed it
        # extract the column vector
        yn = x_vect[M-1+i:i-1:-1]

        R = R + yn @ yn.conj().T

    return R / N
@jit
def wrapper(x):
  return np.convolve(x,xc,mode="valid")

@jit
def esprit1(x: np.ndarray, L: int, M: int=None, fs: int=1,
           verbose: bool=False) -> Tuple[np.ndarray, np.ndarray]:
    """
    This function estimate the frequency components based on the ESPRIT algorithm [ROY89]_
    The frequencies are related to the roots as :math:`z=e^{-2j\pi f/Fe}`.
    See [STO97]_ section 4.7 for more information about the implementation.
    :param x: ndarray, Nsamples
    :param L: int. Number of components to be extracted.
    :param M:  int, optional. Size of signal block.
    :param Fs: float. Sampling Frequency.
    :returns: ndarray ndarray containing the L frequencies
    >>> import numpy as np
    >>> import spectral_analysis.spectral_analysis as sa
    >>> Fe=500
    >>> t=1.*np.arange(100)/Fe
    >>> x=np.exp(2j*np.pi*55.2*t)
    >>> f=sa.Esprit(x,1,None,Fe)
    >>> print(f)
    """

    x = np.asarray(x).squeeze()
    assert x.ndim in (1, 2)
    # length of the vector x
    if x.ndim == 1:
        N = x.size
    else:
        N = x.shape[1]

    if M is None:
        M = N // 2
# %% extract signal subspace  99.9 % of computation time
    tic = time()
    if x.ndim == 1 and isinstance(M, int):
        R = compute_autocovariance(x, M)  # 75% of computation time        
    else:
        # the random phase of transmit/receive/target actually helps--need at least 5-6 observations to make useful
        R = np.cov(x, rowvar=False)
    if verbose:
        print('autocov sec.', time()-tic)
    # R = subspace.corrmtx(x.astype(complex128),M).astype(float) #f2py fortran

    tic = time()
    #U, S, V = lg.svd(R)  # 25% of computation time
    w, v = lg.eig(R)    
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]    
    if verbose:
        print('svd sec.', time()-tic)
# %% take eigenvalues and determine sinusoid frequencies
    # Remove last row
    S1 = v[:-1, :L]
    # Remove first row
    S2 = v[1:, :L]

    # Compute matrix Phi (Stoica 4.7.12)  <0.1 % of computation time
    Phi = lg.inv(S1.conj().T @ S1) @ S1.conj().T @ S2

    # Perform eigenvalue decomposition <0.1 % of computation time
    V, U = lg.eig(Phi)

    # extract frequencies ((note that there a minus sign since Yn are defined as [y(n), y(n-1),y(n-2),..].T))
    ang = -np.angle(V)

    # frequency normalisation
    f = fs*ang / (2.*np.pi)
    t = np.arange(0, 0.01, 1/fs)
    
    x2 = np.exp(-1j*2*np.pi*t[::-1])[:,np.newaxis]**f
    #display(xc.shape)
    #ampCisoid = np.apply_along_axis(lambda q: np.convolve(q,xc,mode="valid"),axis = 0,arr=x2)
    ampCisoid = np.apply_along_axis(wrapper,axis = 0,arr=x2)
    return f, np.abs(w[:L]), np.abs(ampCisoid)*1./480., np.angle(ampCisoid)
  

def esprit2(x: np.ndarray, L: int, M: int=None, fs: int=1,
           verbose: bool=False) -> Tuple[np.ndarray, np.ndarray]:
    """
    This function estimate the frequency components based on the ESPRIT algorithm [ROY89]_
    The frequencies are related to the roots as :math:`z=e^{-2j\pi f/Fe}`.
    See [STO97]_ section 4.7 for more information about the implementation.
    :param x: ndarray, Nsamples
    :param L: int. Number of components to be extracted.
    :param M:  int, optional. Size of signal block.
    :param Fs: float. Sampling Frequency.
    :returns: ndarray ndarray containing the L frequencies
    >>> import numpy as np
    >>> import spectral_analysis.spectral_analysis as sa
    >>> Fe=500
    >>> t=1.*np.arange(100)/Fe
    >>> x=np.exp(2j*np.pi*55.2*t)
    >>> f=sa.Esprit(x,1,None,Fe)
    >>> print(f)
    """

    x = np.asarray(x).squeeze()
    assert x.ndim in (1, 2)
    # length of the vector x
    if x.ndim == 1:
        N = x.size
    else:
        N = x.shape[1]

    if M is None:
        M = N // 2
# %% extract signal subspace  99.9 % of computation time
    tic = time()
    if x.ndim == 1 and isinstance(M, int):
        R = compute_autocovariance(x, M)  # 75% of computation time
    else:
        # the random phase of transmit/receive/target actually helps--need at least 5-6 observations to make useful
        R = np.cov(x, rowvar=False)
    if verbose:
        print('autocov sec.', time()-tic)
    # R = subspace.corrmtx(x.astype(complex128),M).astype(float) #f2py fortran

    tic = time()
    U, S, V = lg.svd(R)  # 25% of computation time
    #w, v = lg.eig(R)    
    #idx = w.argsort()[::-1]   
    #w = w[idx]
    #v = v[:,idx]    
    if verbose:
        print('svd sec.', time()-tic)
# %% take eigenvalues and determine sinusoid frequencies
    # Remove last row
    S1 = U[:-1, :L]
    # Remove first row
    S2 = U[1:, :L]

    # Compute matrix Phi (Stoica 4.7.12)  <0.1 % of computation time
    Phi = lg.inv(S1.conj().T @ S1) @ S1.conj().T @ S2

    # Perform eigenvalue decomposition <0.1 % of computation time
    V, U = lg.eig(Phi)

    # extract frequencies ((note that there a minus sign since Yn are defined as [y(n), y(n-1),y(n-2),..].T))
    ang = -np.angle(V)

    # frequency normalisation
    f = fs*ang / (2.*np.pi)

    return f, S[:L]

In [63]:
from time import time
import numpy as np
from pandas import DataFrame

# Bool: Needs to be set by user 
full_anom = 0
anom_samples = 0

x = np.random.randn(4096).astype(np.complex128)
F = 12345.6

f0 = 12345.6
f1 = 1000.6
f2 = 726.6
f3 = 57.3

fs = 48e3
snr = 20.  # dB
Ntone = 6

t = np.arange(0, 0.01, 1/fs)

nvar = 10**(-snr/10.)



M = [100] * 200  # iterating over block length

py = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fortreal = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fortcmpl = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fest = []
fest2 = []
cisoidAmp = []
cisoidAmp2 = []
i = 0
for m in M:
    i = i + 1
    # Generate fist tripplet of cisoid  
    # Train data start
    xc = 15.*np.exp(1j*2*np.pi*f0*t)+7.*np.exp(1j*2*np.pi*f1*t) +10.*np.exp(1j*2*np.pi*f2*t)
    xc1 = xc + np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
  
    tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc1, Ntone//2, M=m, fs=fs, verbose=False)
    fest.append(tmpFest)
    tmpAmp = tmpAmp[0]
    cisoidAmp.append(tmpAmp)
    # Train data end 
    
    # Test section start
    
    # Inject Anomaly
    if not(full_anom or i%20):
      display(i)
      for j in np.arange(anom_samples):
        display(i)
        xc2 = 15.*np.exp(1j*2*np.pi*f0*t)+7.*np.exp(1j*2*np.pi*f1*t) +10.*np.exp(1j*2*np.pi*f3*t)+ np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
        tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc2, Ntone//2, M=m, fs=fs, verbose=False)
        fest2.append(tmpFest)
        tmpAmp = tmpAmp[0]
        cisoidAmp2.append(tmpAmp) 
    # When not Anomaly use xc as baseline signal and add AWGN - Run through esprit
    xc2 = xc +  np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
    if full_anom:
      xc2 = 15.*np.exp(1j*2*np.pi*f0*t)+7.*np.exp(1j*2*np.pi*f1*t) +10.*np.exp(1j*2*np.pi*f3*t)+ np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
    tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc2, Ntone//2, M=m, fs=fs, verbose=False)
    fest2.append(tmpFest)
    tmpAmp = tmpAmp[0]
    cisoidAmp2.append(tmpAmp)  
    # Test section end 

# Create train and test set     
x_train = np.hstack((cisoidAmp,fest))    
x_test = np.hstack((cisoidAmp2,fest2))
    

20

40

60

80

100

120

140

160

180

200

In [64]:
from keras import regularizers
from sklearn.preprocessing import StandardScaler
# Make sure to run this after each new generation of data
# zero mean and unit var
x_train2 = StandardScaler().fit_transform(x_train)
x_test2 = StandardScaler().fit_transform(x_test)
#x_train2 = x_train2[:,:,np.newaxis]
#x_test2 = x_test2[:,:,np.newaxis]

x_train2 = x_train2.astype('float32')
x_test2 = x_test2.astype('float32') 

display(x_train2.shape)

input_dim = x_train.shape[1]

# set size of autoencoder 
encoding_dim = 6

# use elu because it is leaky tried both net and l1 and l2 : net and l1 worked the best 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="elu",kernel_initializer= 'he_normal', activity_regularizer=regularizers.l1_l2(l1=10e-5, l2=10e-4))(input_layer)
#encoder = Dense(encoding_dim, activation="elu",activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="elu",kernel_initializer= 'he_normal')(encoder)
decoder = Dense(int(encoding_dim / 2), activation='elu',kernel_initializer= 'he_normal')(encoder)
decoder = Dense(input_dim, activation='elu',kernel_initializer= 'he_normal')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.summary()  


(200, 6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_45 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_46 (Dense)             (None, 3)                 21        
_________________________________________________________________
dense_47 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_48 (Dense)             (None, 6)                 24        
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy_with_masking', factor=0.2,
                              patience=5,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

In [66]:
from keras import regularizers
from sklearn.preprocessing import StandardScaler
# Make sure to run this after each new generation of data
# zero mean and unit var
x_train2 = StandardScaler().fit_transform(x_train)
x_test2 = StandardScaler().fit_transform(x_test)
#x_train2 = x_train2[:,:,np.newaxis]
#x_test2 = x_test2[:,:,np.newaxis]

#x_train2 = x_train2.astype('float32') / 255.
#x_test2 = x_test2.astype('float32') / 255.

display(x_train2.shape)

input_dim = x_train.shape[1]

# set size of autoencoder 
encoding_dim = 6
elu = keras.layers.ELU(alpha=1.2)
lrlu= keras.layers.LeakyReLU(alpha=0.3)

xavier = keras.initializers.glorot_uniform(seed=None)



# use elu because it is leaky tried both net and l1 and l2 : net and l1 worked the best 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim,kernel_initializer= xavier, activity_regularizer=regularizers.l1_l2(l1=10e-5, l2=75.10e-5))(input_layer)
encoder = keras.layers.LeakyReLU(alpha=0.011)(encoder)
#encoder = Dense(encoding_dim, activation="elu",activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2),kernel_initializer= xavier)(encoder)
encoder = keras.layers.LeakyReLU(alpha=0.011)(encoder)
decoder = Dense(int(encoding_dim / 2),kernel_initializer=xavier)(encoder)
decoder = keras.layers.LeakyReLU(alpha=0.011)(decoder)
decoder = Dense(input_dim,kernel_initializer=xavier)(decoder)
decoder = keras.layers.LeakyReLU(alpha=0.011)(decoder)
autoencoder2 = Model(inputs=input_layer, outputs=decoder)

autoencoder2.summary()  


(200, 6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 6)                 0         
_________________________________________________________________
dense_49 (Dense)             (None, 6)                 42        
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 6)                 0         
_________________________________________________________________
dense_50 (Dense)             (None, 3)                 21        
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 3)                 0         
_________________________________________________________________
dense_51 (Dense)             (None, 3)                 12        
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 3)                 0         
__________

In [0]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 16
import keras

nadam =keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, schedule_decay=0.004)

autoencoder2.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder2.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder2.evaluate(x_test2, x_test2, batch_size=16, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 2s 12ms/step - loss: 0.5050 - acc: 0.6833 - val_loss: 0.6845 - val_acc: 0.5000
Epoch 2/800
180/180 [==============================] - 0s 602us/step - loss: 0.5022 - acc: 0.7333 - val_loss: 0.6829 - val_acc: 0.5500
Epoch 3/800
180/180 [==============================] - 0s 609us/step - loss: 0.5020 - acc: 0.7167 - val_loss: 0.6836 - val_acc: 0.5000
Epoch 4/800
180/180 [==============================] - 0s 640us/step - loss: 0.5001 - acc: 0.7333 - val_loss: 0.6790 - val_acc: 0.4500
Epoch 5/800
180/180 [==============================] - 0s 620us/step - loss: 0.4987 - acc: 0.7333 - val_loss: 0.6791 - val_acc: 0.5500
Epoch 6/800
180/180 [==============================] - 0s 599us/step - loss: 0.4973 - acc: 0.7222 - val_loss: 0.6835 - val_acc: 0.5500
Epoch 7/800
180/180 [==============================] - 0s 688us/step - loss: 0.4991 - acc: 0.7333 - val_loss: 0.6831 - val_acc: 0.5000
Epoch 8/800

'[test_loss, test_acc]'

[0.3951062059402466, 0.735]

In [0]:
#@title Train and Test are anomoly free
compile = "loss: MSE --  metric: acc" #@param {type:"string"}
fit = "trains and validates on x_train2 " #@param {type:"string"}
evaluate = "returns a score =[test_loss and an test_acc] using data set x_test2" #@param {type:"string"}
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 16
import keras

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
nadam =keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

autoencoder2.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder2.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder2.evaluate(x_test2, x_test2, batch_size=16, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 2s 11ms/step - loss: 0.4230 - acc: 0.8167 - val_loss: 0.5745 - val_acc: 0.7500
Epoch 2/800
180/180 [==============================] - 0s 620us/step - loss: 0.4159 - acc: 0.8500 - val_loss: 0.5679 - val_acc: 0.8500
Epoch 3/800
180/180 [==============================] - 0s 610us/step - loss: 0.4161 - acc: 0.8333 - val_loss: 0.5752 - val_acc: 0.8000
Epoch 4/800
180/180 [==============================] - 0s 601us/step - loss: 0.4146 - acc: 0.8167 - val_loss: 0.5759 - val_acc: 0.7000
Epoch 5/800
180/180 [==============================] - 0s 618us/step - loss: 0.4161 - acc: 0.8333 - val_loss: 0.5908 - val_acc: 0.8000
Epoch 6/800
180/180 [==============================] - 0s 631us/step - loss: 0.4190 - acc: 0.8222 - val_loss: 0.5768 - val_acc: 0.6500
Epoch 7/800
180/180 [==============================] - 0s 635us/step - loss: 0.4207 - acc: 0.8389 - val_loss: 0.5911 - val_acc: 0.7000
Epoch 8/800

'[test_loss, test_acc]'

[0.4151963233947754, 0.955]

In [69]:
#@title Train and Test are anomoly free
compile = "loss: MSE --  metric: acc" #@param {type:"string"}
fit = "trains and validates on x_train2 " #@param {type:"string"}
evaluate = "returns a score =[test_loss and an test_acc] using data set x_test2" #@param {type:"string"}
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 16
import keras

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
nadam =keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

autoencoder2.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder2.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder2.evaluate(x_test2, x_test2, batch_size=16, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 3s 14ms/step - loss: 0.4297 - acc: 0.8333 - val_loss: 0.4668 - val_acc: 0.9000
Epoch 2/800
180/180 [==============================] - 0s 624us/step - loss: 0.4241 - acc: 0.8667 - val_loss: 0.4663 - val_acc: 0.9500
Epoch 3/800
180/180 [==============================] - 0s 576us/step - loss: 0.4207 - acc: 0.8778 - val_loss: 0.5262 - val_acc: 0.8000
Epoch 4/800
180/180 [==============================] - 0s 576us/step - loss: 0.4320 - acc: 0.8389 - val_loss: 0.4694 - val_acc: 0.9500
Epoch 5/800
180/180 [==============================] - 0s 580us/step - loss: 0.4220 - acc: 0.8611 - val_loss: 0.4712 - val_acc: 0.9500
Epoch 6/800
180/180 [==============================] - 0s 573us/step - loss: 0.4268 - acc: 0.8556 - val_loss: 0.4764 - val_acc: 0.8000
Epoch 7/800
180/180 [==============================] - 0s 588us/step - loss: 0.4313 - acc: 0.8444 - val_loss: 0.4726 - val_acc: 0.9000
Epoch 8/800

'[test_loss, test_acc]'

[0.46604501008987426, 0.95]

In [0]:
#@title Train and Test are anomoly free
compile = "loss: MSE --  metric: acc" #@param {type:"string"}
fit = "trains and validates on x_train2 " #@param {type:"string"}
evaluate = "returns a score =[test_loss and an test_acc] using data set x_test2" #@param {type:"string"}
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 16
import keras

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
nadam =keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

autoencoder2.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder2.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder2.evaluate(x_test2, x_test2, batch_size=16, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

In [0]:
#@title Train set is anomoly free Test set has anomoly 

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 0s 2ms/step - loss: 0.1171 - acc: 0.8222 - val_loss: 0.0864 - val_acc: 0.6500
Epoch 2/800
180/180 [==============================] - 0s 300us/step - loss: 0.1157 - acc: 0.8056 - val_loss: 0.0879 - val_acc: 0.6500
Epoch 3/800
180/180 [==============================] - 0s 267us/step - loss: 0.1151 - acc: 0.8333 - val_loss: 0.0861 - val_acc: 0.6500
Epoch 4/800
180/180 [==============================] - 0s 273us/step - loss: 0.1154 - acc: 0.8167 - val_loss: 0.0865 - val_acc: 0.6500
Epoch 5/800
180/180 [==============================] - 0s 259us/step - loss: 0.1151 - acc: 0.8333 - val_loss: 0.0873 - val_acc: 0.6500
Epoch 6/800
180/180 [==============================] - 0s 256us/step - loss: 0.1154 - acc: 0.8278 - val_loss: 0.0878 - val_acc: 0.6500
Epoch 7/800
180/180 [==============================] - 0s 274us/step - loss: 0.1162 - acc: 0.8222 - val_loss: 0.0876 - val_acc: 0.6500
Epoch 8/800


'[test_loss, test_acc]'

[2.6125407600402832, 0.27]

In [0]:
#@title Train set is anomoly free Test set has anomoly 

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.predict(x_test2, batch_size=32, verbose=1, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 0s 2ms/step - loss: 0.1126 - acc: 0.8778 - val_loss: 0.1063 - val_acc: 0.9500
Epoch 2/800
180/180 [==============================] - 0s 248us/step - loss: 0.1082 - acc: 0.9000 - val_loss: 0.1044 - val_acc: 0.9000
Epoch 3/800
180/180 [==============================] - 0s 251us/step - loss: 0.1072 - acc: 0.9056 - val_loss: 0.1066 - val_acc: 0.9000
Epoch 4/800
180/180 [==============================] - 0s 249us/step - loss: 0.1083 - acc: 0.8889 - val_loss: 0.1045 - val_acc: 0.9000
Epoch 5/800
180/180 [==============================] - 0s 246us/step - loss: 0.1095 - acc: 0.8889 - val_loss: 0.1065 - val_acc: 0.9000
Epoch 6/800
180/180 [==============================] - 0s 231us/step - loss: 0.1076 - acc: 0.8833 - val_loss: 0.1063 - val_acc: 0.8500
Epoch 7/800
180/180 [==============================] - 0s 241us/step - loss: 0.1080 - acc: 0.8722 - val_loss: 0.1041 - val_acc: 0.8500
Epoch 8/800


'[test_loss, test_acc]'

array([[ 6.1436176 , -0.9054951 , -0.61835647,  6.5494404 ,  2.087459  ,
        -0.6207515 ],
       [ 5.7686443 , -0.9523954 , -0.8084736 ,  6.148201  ,  2.7449708 ,
        -0.81013155],
       [ 1.4915143 , -0.4228766 , -0.22654045,  1.5473499 ,  0.24837255,
        -0.22921228],
       ...,
       [-0.68654233,  3.7373142 , -0.8782671 , -0.7289171 , -0.9809346 ,
        -0.8794309 ],
       [-0.6065277 ,  4.709182  , -0.99035704, -0.65331614, -0.99256074,
        -0.99053323],
       [ 2.7909532 , -0.9759932 , -0.80574316,  2.9493818 ,  3.354622  ,
        -0.8073797 ]], dtype=float32)

In [0]:
score.shape

(200, 6)

In [0]:
#Results with one sample anomily 
# the duration of the anom can be tuned with anom_samples
full_anom = 0
anom_samples = 1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 0s 3ms/step - loss: 0.1165 - acc: 0.8667 - val_loss: 0.0725 - val_acc: 0.9500
Epoch 2/800
180/180 [==============================] - 0s 296us/step - loss: 0.1121 - acc: 0.9056 - val_loss: 0.0745 - val_acc: 0.9500
Epoch 3/800
180/180 [==============================] - 0s 300us/step - loss: 0.1127 - acc: 0.8667 - val_loss: 0.0772 - val_acc: 0.9000
Epoch 4/800
180/180 [==============================] - 0s 285us/step - loss: 0.1128 - acc: 0.8833 - val_loss: 0.0724 - val_acc: 0.9500
Epoch 5/800
180/180 [==============================] - 0s 321us/step - loss: 0.1133 - acc: 0.8833 - val_loss: 0.0734 - val_acc: 1.0000
Epoch 6/800
180/180 [==============================] - 0s 339us/step - loss: 0.1134 - acc: 0.8833 - val_loss: 0.0693 - val_acc: 1.0000
Epoch 7/800
180/180 [==============================] - 0s 310us/step - loss: 0.1128 - acc: 0.8722 - val_loss: 0.0739 - val_acc: 0.9500
Epoch 8/800


'[test_loss, test_acc]'

[1.0017147807847886, 0.4142857151372092]